### make sure to install
```
pip install ipympl
pip install nodejs
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install jupyter-matplotlib
```

credit: https://stackoverflow.com/a/56416229/11756613

In [ ]:
%load_ext lab_black
%matplotlib widget
# %matplotlib nbagg should support double clicks, but doesn't work in jupyter :(

import pickle
import time
import os
from glob import glob
from datetime import datetime
from queue import LifoQueue

import numpy as np
import networkx as nx
import ipywidgets as widgets
import matplotlib.pyplot as plt

plt.style.use("dark_background")

In [ ]:
# temporary autocompletion bug fix
%config Completer.use_jedi = False

In [ ]:
def poly(sides, G, n1, n2):
    #     # ensure that they are connected
    #     if (n1, n2) not in G.edges and (n2, n1) not in G.edges and len(G.edges) > 0:
    #         return
    # save before adding a polygon
    history.put(G.copy())

    ns = [n1, n2]
    poss = []
    poss.append(G.nodes[n1]["pos"])
    poss.append(G.nodes[n2]["pos"])
    direction = poss[1] - poss[0]
    # assert np.isclose(1, abs(direction), atol=0.5)

    rotation = np.exp(2 * np.pi * 1j / sides)
    for i in range(1, sides - 1):
        poss.append(poss[-1] + direction * rotation ** i)

    existing_poss = nx.get_node_attributes(G, "pos")

    for pos in poss[2:]:
        match = [
            node
            for node, e_pos in existing_poss.items()
            if np.isclose(e_pos, pos, atol=tolerance_slider.value)
        ]
        if match:
            # node already exists in this position, so use it
            assert len(match) == 1
            ns.append(match[0])
        else:
            # no node exists in this position, so create it
            ns.append(len(G.nodes))  # assumes nodes are numbered 0,1,2,...
            G.add_node(ns[-1], pos=pos)

    for i in range(sides):
        # fmt: off
        G.add_edge(
            ns[i], ns[(i + 1) % sides], 
            angle=rotation,                   # it's the angle that should be at the second node
            next_vertex=ns[(i + 2) % sides]   # handle for the next vertex of this polygon
        )
        # fmt: on


def even_out(
    _,
    step_size=0.1,
    constant_side_length=False,
    stop_on=0.003,
    quadratic_corrections=False,
    max_iter=200,
):
    # even out the angles
    global G, ax, out

    for i in range(max_iter):
        # calculate corrections
        for node in G.nodes:
            G.nodes[node]["correction"] = 0

        loss = 0
        for edge in G.edges:
            if "next_vertex" in G.edges[edge]:
                n1, n2 = edge
                n3 = G.edges[edge]["next_vertex"]
                angle = G.edges[edge]["angle"]
                # note, that it is an external, not an internal angle
                pos1 = G.nodes[n1]["pos"]
                pos2 = G.nodes[n2]["pos"]
                pos3 = G.nodes[n3]["pos"]

                # calculate correction for n3 and add it to accumulator
                side = pos2 - pos1
                if constant_side_length:
                    side /= abs(pos2 - pos1)
                    # normalization ensures sides tend to 1
                desired_pos3 = pos2 + side * angle
                correction = (desired_pos3 - pos3) / 2
                if quadratic_corrections:
                    correction *= min(abs(correction), 0.3)  # min to prevent blowing up
                    # this squaring of the force (with abs) is needed
                    # to penalize biggest errors the most
                G.nodes[n3]["correction"] = correction + G.nodes[n3]["correction"]
                # apply opposite force in the middle node
                G.nodes[n2]["correction"] = -correction + G.nodes[n2]["correction"]

                # same for n1
                side = pos2 - pos3
                if constant_side_length:
                    side /= abs(pos2 - pos3)
                desired_pos1 = pos2 + side / angle
                correction = (desired_pos1 - pos1) / 2
                if quadratic_corrections:
                    correction *= min(abs(correction), 0.3)  # min to prevent blowing up
                G.nodes[n1]["correction"] = correction + G.nodes[n1]["correction"]
                # apply opposite force in the middle node
                G.nodes[n2]["correction"] = -correction + G.nodes[n2]["correction"]

        corrections = nx.get_node_attributes(G, "correction")
        max_corr = max(abs(val) for val in corrections.values())
        # print(max_corr)
        if max_corr < stop_on:
            with out:
                out.clear_output()
                print(i, "iterations")
            break

        # apply corrections
        for node in G.nodes:
            pos = G.nodes[node]["pos"]
            correction = G.nodes[node]["correction"]
            G.nodes[node]["pos"] = pos + correction * step_size

    redraw(_)


def outside_graph(G):
    # find edges on the outside
    # inside edges always come in pairs (a, b), (b, a)
    unpaired_edges = G.edges ^ G.reverse().edges
    return G.edge_subgraph(unpaired_edges)

In [ ]:
def redraw(_, clear=True):
    if clear:
        ax.clear()
        size = size_selector.value
        ax.set_xlim([-size, size])
        ax.set_ylim([-size, size])

    positions = nx.get_node_attributes(G, "pos")
    # convert complex position to a 2D tuple
    positions = {key: (value.real, value.imag) for key, value in positions.items()}
    nx.draw_networkx(
        G.to_undirected(),
        positions,
        ax,
        with_labels=False,
        node_size=0,
        edge_color="white",
    )

    if completions_tickbox.value:
        draw_completions()


def initialize_graph(_):
    global G, history
    G = nx.DiGraph()
    G.add_node(0, pos=0)
    G.add_node(1, pos=1)

    history = LifoQueue()
    redraw(_)


def completions():
    Outside = outside_graph(G)

    labels = dict()

    for node in Outside.nodes:
        predecessor = list(Outside.predecessors(node))[0]
        successor = list(Outside.successors(node))[0]

        pos0 = Outside.nodes[predecessor]["pos"]
        pos1 = Outside.nodes[node]["pos"]
        pos2 = Outside.nodes[successor]["pos"]

        angle_complex = (pos2 - pos1) / (pos0 - pos1)
        angle_radians = np.real(np.log(angle_complex) / 1j)
        angle_degrees = angle_radians / (2 * np.pi) * 360

        if 1 <= angle_degrees < 75:
            completion = 3
        elif 75 <= angle_degrees < 99:
            completion = 4
        elif 99 <= angle_degrees < 114:
            completion = 5
        elif 114 <= angle_degrees < 124.3:
            completion = 6
        elif 124.3 <= angle_degrees < 131.8:
            completion = 7
        elif 131.8 <= angle_degrees < 138:
            completion = 8
        else:
            continue

        labels[node] = completion
    return labels


def draw_completions():
    labels = completions()
    S = G.subgraph(labels.keys())
    positions = nx.get_node_attributes(G, "pos")
    # convert complex position to a 2D tuple
    positions = {key: (value.real, value.imag) for key, value in positions.items()}
    nx.draw_networkx(
        S.to_undirected(),
        positions,
        ax,
        with_labels=True,
        labels=labels,
        node_size=210,
        edge_color="white",
    )


def onclick(event):
    global G, ax
    click_pos = event.xdata + event.ydata * 1j
    positions = nx.get_node_attributes(G, "pos")
    positions = list(positions.items())
    positions.sort(key=lambda pair: abs(pair[1] - click_pos))
    n1, pos1 = positions[0]
    n2, pos2 = positions[1]

    # find on which side the click was
    side = ((click_pos - pos1) / (pos2 - pos1)).imag
    if 0 > side:
        # if it's the bad side, flip nodes
        n1, n2 = n2, n1

    # event.dblclick try outside of jupyter with nbagg
    if event.button == 1:  # left click
        # draw chosen polygon
        sides = polygon_options[polygon_selector.value]
        poly(sides, G, n1, n2)
    elif event.button == 3:  # right click
        # choose next polygon
        polygon_selector.value = (polygon_selector.value + 1) % len(polygon_options)

    redraw(_)
    # even_out(_)


def undo(_):
    global G, history, ax
    if history.qsize() > 0:
        G = history.get()
        redraw(_)


def load(name):
    global G
    name = name["new"]
    with open(f"saves/{name}", "rb") as handle:
        G = pickle.load(handle)
    redraw(_)


def save(_):
    name = save_namebox.value
    timestamp = datetime.utcnow().strftime("%Y-%m-%d %H-%M-%S")
    with open(f"saves/{timestamp} {name}", "wb") as handle:
        pickle.dump(G, handle, protocol=pickle.HIGHEST_PROTOCOL)
    load_selector.options = get_saves()


def get_saves():
    saves = glob("saves/*")
    saves = sorted(saves, key=os.path.getmtime, reverse=True)
    saves = [save.split("/")[-1] for save in saves]
    return saves

In [ ]:
# define figure
fig, ax = plt.subplots()
canvas_size = 7
fig.set_size_inches(canvas_size, canvas_size)
fig.tight_layout()

polygon_options = [3, 4, 5, 6, 7, 8]
polygon_selector = widgets.Dropdown(
    options=[(poly, i) for i, poly in enumerate(polygon_options)],
    value=0,
    description="Sides: ",
)

cid = fig.canvas.mpl_connect("button_press_event", onclick)


# create and bind all widgets
out = widgets.Output()
undo_button = widgets.Button(description="undo")
undo_button.on_click(undo)
even_out_button = widgets.Button(description="even out")
even_out_button.on_click(even_out)
load_selector = widgets.Dropdown(options=get_saves(), description="Load: ")
load_selector.observe(load, names="value")
clear_button = widgets.Button(description="clear")
clear_button.on_click(initialize_graph)
completions_tickbox = widgets.Checkbox(value=False, description="Completions")
completions_tickbox.observe(redraw, names="value")
size_selector = widgets.FloatLogSlider(
    value=10, base=10, min=0, max=4, step=0.05, description="Size:"
)
size_selector.observe(redraw, names="value")
save_namebox = widgets.Text(description="Save name: ", value="save")
save_button = widgets.Button(description="save")
save_button.on_click(save)
tolerance_slider = widgets.FloatSlider(
    value=0.1, min=0, max=1, description="Tolerance:"
)


initialize_graph(_)

widgets.VBox(
    [
        widgets.HBox([polygon_selector, undo_button, even_out_button, out]),
        widgets.HBox([load_selector, clear_button, tolerance_slider]),
        widgets.HBox([save_namebox, save_button, size_selector]),
        widgets.HBox([completions_tickbox]),
    ]
)